# Singular Value Decomposition with Numpy and SciPy - Lab

## Introduction

In this lab, we will build a basic version of low-rank matrix factorization engine for recommending movies in using Numpy and SciPy. The purpose of this lesson to help develop a sound understanding of how this process works in detail, before we implement it in PySpark. We will use a small movie recommendation dataset suitable for such experiments. The system should make not highly accurate, yet meaningful recommendations. 

## Objectives

You will be able to:

- Build a basic recommendation system using MovieLense 1 million movies dataset
- Use Scipy and Numpy to build a recommendation system using matrix factorization. 


## Dataset
For this lab, we will use  a dataset of 1 million movie ratings  available from the [MovieLens](http://grouplens.org/datasets/movielens/) project collected by GroupLens Research at the University of Minnesota. The website offers many versions and subsets of the complete MovieLens dataset. We have downloaded the 1 million movies subset for you and you can find it under the folder `ml-1m`. [Visit this link](http://files.grouplens.org/datasets/movielens/ml-latest-README.html) and also the MovieLens site above to get an understanding of format for included files before moving on:

- ratings.dat
- users.dat
- movies.dat

Let's first read our dataset into pandas dataframe before moving on. 

Our datasets are `.dat` format with features split using the delimiter `'::'`. Perform following tasks:

- Read the files `ratings.dat`, `movies.dat` and `users.dat` using python's `open()`. Use `encoding='latin-1'` for these files
- Split above files on delimiter '::' and create arrays for users , movies and ratings

- Create ratings and movies dataframes from arrays above with columns:
    - `ratings = ['UID', 'MID', 'Rating', 'Time']`
    - `movies = ['MID', 'Title', 'Genre']`
- View the contents of `movies` and `ratings` datasets

*Note: Make sure to change the appropriate datatypes to int (numeric) in these datasets.*

In [1]:
import pandas as pd
import numpy as np

# Cerate movies, ratings and users dataframes (reading files, splitting, creating arrays and dataframes)
ratings = pd.DataFrame(np.array([i.strip().split("::") for i in open('ml-1m/ratings.dat', 'r', encoding='latin-1').readlines()]),
                       columns = ['UID', 'MID', 'Rating', 'Time'], dtype = int)
movies = pd.DataFrame(np.array([i.strip().split("::") for i in open('ml-1m/movies.dat', 'r',  encoding='latin-1').readlines()]),
                         columns = ['MID', 'Title', 'Genre'])
movies['MID'] = movies['MID'].apply(pd.to_numeric)


users= np.array([i.strip().split("::") for i in open('ml-1m/users.dat', 'r',  encoding='latin-1').readlines()])


In [2]:
# Uncomment Below

print(movies.head())
print()
print(ratings.head())

   MID                               Title                         Genre
0    1                    Toy Story (1995)   Animation|Children's|Comedy
1    2                      Jumanji (1995)  Adventure|Children's|Fantasy
2    3             Grumpier Old Men (1995)                Comedy|Romance
3    4            Waiting to Exhale (1995)                  Comedy|Drama
4    5  Father of the Bride Part II (1995)                        Comedy

   UID   MID  Rating       Time
0    1  1193       5  978300760
1    1   661       3  978302109
2    1   914       3  978301968
3    1  3408       4  978300275
4    1  2355       5  978824291


## Creating the Utility Matrix 

Matrix factorization, as we saw in the previous lesson, uses a "Utility Matrix" of users x movies. The intersection points between users and movies indicate the ratings users have given to movies. We saw how this is mostly sparse matrix. Here is a quick refresher below:

<img src="utility.jpg" width=400>

Next, our job is to create such a matrix from the ratings table above in order to proceed forward with SVD. 

- Create a utility matrix `A` to contain one row per user and one column per movie. Pivot `ratings` dataframe to achieve this. 

In [2]:
# Create a utility matrix A by pivoting ratings.df
A = ratings.pivot(index = 'UID', columns ='MID', values = 'Rating').fillna(0)
A.head()

MID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
A.shape

(6040, 3706)

Finally, let's perform mean normalization on our utility matrix, and save it as a numpy array for decomposition tasks.

In [3]:

# MEan normalize dataframe A to numpy array A_norm
A_vals = A.values
ratings_mean = np.mean(A_vals, axis = 1)
A_norm = A_vals - ratings_mean.reshape(-1, 1)
A_norm[:10]

array([[ 4.94009714, -0.05990286, -0.05990286, ..., -0.05990286,
        -0.05990286, -0.05990286],
       [-0.12924987, -0.12924987, -0.12924987, ..., -0.12924987,
        -0.12924987, -0.12924987],
       [-0.05369671, -0.05369671, -0.05369671, ..., -0.05369671,
        -0.05369671, -0.05369671],
       ...,
       [ 3.85429034, -0.14570966, -0.14570966, ..., -0.14570966,
        -0.14570966, -0.14570966],
       [ 4.89314625, -0.10685375, -0.10685375, ..., -0.10685375,
        -0.10685375, -0.10685375],
       [ 4.55477604,  4.55477604, -0.44522396, ..., -0.44522396,
        -0.44522396, -0.44522396]])

In [26]:
ratings_mean

array([0.05990286, 0.12924987, 0.05369671, ..., 0.02050729, 0.1287102 ,
       0.3291959 ])

## Matrix Factorization with SVD

SVD can help us decomposes the matrix $A$ into the best lower rank approximation of the original matrix $A$. Mathematically, it decomposes $A$ into a two unitary matrices and a diagonal matrix:

$$\begin{equation}
A = U\Sigma V^{T}
\end{equation}$$

A above is users' ratings matrix (utility), $U$ is the user "features" matrix, $\Sigma$ is the diagonal matrix of singular values (essentially weights), and $V^{T}$ is the movie "features" matrix. $U$ and $V^{T}$ are orthogonal, and represent different things. $U$ represents how much users "like" each feature and $V^{T}$ represents how relevant each feature is to each movie.

To get the lower rank approximation, we take these matrices and keep only the top $k$ features, which we think of as the underlying tastes and preferences vectors.

Perform following tasks:
- Import `svds` from Scipy
- decompose A_norm using 50 factors i.e. pass `k=50` argument to `svds()`


In [4]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(A_norm, k = 50)

In [10]:
U

array([[-4.97801875e-03,  5.86971868e-03, -1.18186843e-02, ...,
        -2.95139320e-03,  1.95703358e-03,  5.46889776e-03],
       [-1.10526375e-03, -4.04545890e-03,  1.16776791e-02, ...,
        -9.18855171e-04, -2.17034433e-03,  1.04359614e-02],
       [ 9.44963839e-03, -1.43519545e-02, -3.93638119e-04, ...,
         2.89764529e-03, -2.86504507e-03,  6.13985002e-03],
       ...,
       [-1.05731072e-02, -6.80807641e-03, -2.63392883e-03, ...,
         4.84286245e-05,  1.89077440e-03,  1.52456048e-03],
       [ 6.34420788e-03, -9.45269844e-03,  2.69929558e-03, ...,
         1.07208981e-02,  1.88878158e-02,  6.87143535e-03],
       [-1.84854679e-02,  1.28388950e-02,  8.46988257e-03, ...,
         1.89987575e-03,  4.15563933e-02,  1.92850979e-02]])

In [11]:
sigma

array([ 147.18581225,  147.62154312,  148.58855276,  150.03171353,
        151.79983807,  153.96248652,  154.29956787,  154.54519202,
        156.1600638 ,  157.59909505,  158.55444246,  159.49830789,
        161.17474208,  161.91263179,  164.2500819 ,  166.36342107,
        166.65755956,  167.57534795,  169.76284423,  171.74044056,
        176.69147709,  179.09436104,  181.81118789,  184.17680849,
        186.29341046,  192.15335604,  192.56979125,  199.83346621,
        201.19198515,  209.67692339,  212.55518526,  215.46630906,
        221.6502159 ,  231.38108343,  239.08619469,  244.8772772 ,
        252.13622776,  256.26466285,  275.38648118,  287.89180228,
        315.0835415 ,  335.08085421,  345.17197178,  362.26793969,
        415.93557804,  434.97695433,  497.2191638 ,  574.46932602,
        670.41536276, 1544.10679346])

In [12]:
Vt

array([[-0.07028629,  0.02415349, -0.01883837, ...,  0.00380736,
        -0.00049127,  0.00061123],
       [ 0.03681506,  0.00346263, -0.01264234, ..., -0.00965995,
        -0.00513455, -0.02377963],
       [ 0.03495646,  0.00904907,  0.00823098, ...,  0.00157338,
        -0.00234513,  0.00802561],
       ...,
       [-0.03287652,  0.01185799, -0.01107445, ..., -0.00114772,
        -0.00294575, -0.02222119],
       [-0.01776333, -0.03068092, -0.01786526, ...,  0.00087071,
         0.0012666 ,  0.00435186],
       [ 0.07625855,  0.01650222,  0.00468327, ..., -0.00852744,
        -0.01020778,  0.00425656]])

### Creating diagonal matrix for sigma factors

The sigma above is returned as a 1 dimensional array of latent factor values. As we need to perform matrix multiplication in our next part, lets convert it to a diagonal matrix using `np.diag()`. [Here is an explanation for this](https://math.stackexchange.com/questions/1731183/convert-vector-into-diagonal-matrix)

- Convert sigma factors into a diagonal matrix with `np.diag()`
- Check and confirm the shape of `sigma` befora and after conversion 

In [5]:
sigma = np.diag(sigma)

Excellent, We changed sigma from a vector of size fifty to a 2D diagonal matrix of size 50x50. We can now move on to making predictions from our decomposed matrices. 

## Making Predictions from the Decomposed Matrices

Now we have everything required to make movie ratings predictions for every user. We will do it all at once by following the math and matrix multiply $U$, $\Sigma$, and $V^{T}$ back to get the rank $k=50$ __approximation of $A$__. Perform following tasks to achieve this

* Use `np.dot()` to multiply $U,\Sigma, V$
* add the user ratings means back to get the actual __star__ ratings prediction.

In [6]:
predictions = np.dot(np.dot(U, sigma), Vt) + ratings_mean.reshape(-1, 1)

#### For a practical system, the value of `k` above would be identified through creating test and training datasets and selecting optimal value of this parameter. We will leave this bit for our detailed experiment in the next lab. 

Here, we'll see how to make recommendations based on `predictions` array created above.



### Making Recommendations 

With the predictions matrix for every user, we can build a function to recommend movies for any user. We need to return the movies with the highest predicted rating that the specified user hasn't already rated. We will also merge in the user information to get a more complete picture of the recommendations. We will also return the list of movies the user has already rated, for the sake of comparison.
- Create a Dataframe from predictions and view contents
- Use column names of `A` as the new names for this dataframe

In [7]:
predictions_df = pd.DataFrame(predictions, columns = A.columns)
predictions_df.head()

MID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.288861,0.143055,-0.195080,-0.018843,0.012232,-0.176604,-0.074120,0.141358,-0.059553,-0.195950,...,0.027807,0.001640,0.026395,-0.022024,-0.085415,0.403529,0.105579,0.031912,0.050450,0.088910
1,0.744716,0.169659,0.335418,0.000758,0.022475,1.353050,0.051426,0.071258,0.161601,1.567246,...,-0.056502,-0.013733,-0.010580,0.062576,-0.016248,0.155790,-0.418737,-0.101102,-0.054098,-0.140188
2,1.818824,0.456136,0.090978,-0.043037,-0.025694,-0.158617,-0.131778,0.098977,0.030551,0.735470,...,0.040481,-0.005301,0.012832,0.029349,0.020866,0.121532,0.076205,0.012345,0.015148,-0.109956
3,0.408057,-0.072960,0.039642,0.089363,0.041950,0.237753,-0.049426,0.009467,0.045469,-0.111370,...,0.008571,-0.005425,-0.008500,-0.003417,-0.083982,0.094512,0.057557,-0.026050,0.014841,-0.034224
4,1.574272,0.021239,-0.051300,0.246884,-0.032406,1.552281,-0.199630,-0.014920,-0.060498,0.450512,...,0.110151,0.046010,0.006934,-0.015940,-0.050080,-0.052539,0.507189,0.033830,0.125706,0.199244


Now we have a predictions dataframe composed from a reduced factors. We can now can predicted recommendations for every user. We will create a new function `recommender()` as shown below: 

- `recommender()`
    - Inputs: predictions dataframe , chosen UserID, movies dataframe, original ratings df, num_recommendations
    - Outputs: Movies already rated by user, predicted ratings for remaining movies for user

- Get and set of predictions for selected user and sort in descending order
- Get the movies already rated by user and sort them in descending order by rating
- Create a set of recommendations for all movies, not yet rated by the user

In [8]:
# Recommending top movies not yet rated by user
def recommender(predictions_df, UID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row = UID - 1 # UID starts at 1, not 0
    sorted_predictions = predictions_df.iloc[user_row].sort_values(ascending=False) 
    
    # Get the original user data and merge in the movie information 
    user_data = original_ratings_df[original_ratings_df.UID == (UID)]
    user_full = user_data.merge(movies_df, how = 'left', left_on = 'MID', right_on = 'MID').sort_values(['Rating'], ascending=False)

    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = movies_df[~movies_df['MID'].isin(user_full['MID'])] \
                      .merge(pd.DataFrame(sorted_predictions).reset_index(), 
                             how = 'left', left_on = 'MID', right_on = 'MID') \
                      .rename(columns = {user_row: 'Predictions'}) \
                      .sort_values('Predictions', ascending = False) \
                      .iloc[:num_recommendations, :-1]
                    
    print ('User {0} has already rated {1} movies.'.format(UID, user_full.shape[0]))
    print ('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    return user_full, recommendations

In [12]:
    # Get and sort the user's predictions
    user_row = 100 - 1 # UID starts at 1, not 0
    sorted_predictions = predictions_df.iloc[user_row].sort_values(ascending=False)

In [14]:
sorted_predictions

MID
1196    4.544500
260     4.441089
1198    4.392833
2028    3.993405
1210    3.954229
527     3.627267
858     3.154890
1291    2.678817
318     2.673813
1221    2.522624
1197    2.449484
2628    2.406659
110     2.356688
1356    2.273795
919     2.205438
593     2.172561
1374    2.111930
480     2.105087
608     2.048725
1193    1.984488
329     1.973312
1376    1.927722
1580    1.806173
1372    1.796883
296     1.775893
1213    1.741092
2115    1.721880
2571    1.584744
648     1.576152
592     1.527694
          ...   
3098   -0.270821
965    -0.271413
2139   -0.271660
3481   -0.273201
1302   -0.293909
1333   -0.298616
2580   -0.298945
2294   -0.300809
3039   -0.306743
2395   -0.321332
1103   -0.337658
3361   -0.338058
2150   -0.340360
1748   -0.341963
1274   -0.347094
2761   -0.356469
741    -0.362944
47     -0.378527
1246   -0.391419
1625   -0.393485
2997   -0.396578
2542   -0.397834
2762   -0.398402
2915   -0.405036
1965   -0.406190
3052   -0.413578
2959   -0.507611
293    -0.

In [18]:
# Get the original user data and merge in the movie information 
user_data = ratings[ratings.UID == 100]
user_full = user_data.merge(movies, how = 'left', left_on = 'MID', right_on = 'MID').sort_values(['Rating'], ascending=False)

In [21]:
user_data.head()

,UID,MID,Rating,Time
12900,100,648,2,977594265
12901,100,800,5,977593915
12902,100,3948,3,977593808
12903,100,1408,3,977594113
12904,100,1196,4,977593950


In [22]:
movies.head()

,MID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
user_full

,UID,MID,Rating,Time,Title,Genre
1,100,800,5,977593915,Lone Star (1996),Drama|Mystery
63,100,527,5,977594839,Schindler's List (1993),Drama|War
16,100,919,5,977594947,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical
17,100,924,4,977594873,2001: A Space Odyssey (1968),Drama|Mystery|Sci-Fi|Thriller
29,100,969,4,977594044,"African Queen, The (1951)",Action|Adventure|Romance|War
22,100,2406,4,977594142,Romancing the Stone (1984),Action|Adventure|Comedy|Romance
47,100,318,4,977594839,"Shawshank Redemption, The (1994)",Drama
20,100,858,4,977593950,"Godfather, The (1972)",Action|Crime|Drama
49,100,329,4,977594297,Star Trek: Generations (1994),Action|Adventure|Sci-Fi
50,100,260,4,977593595,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi


In [24]:
# Recommend the highest predicted rating movies that the user hasn't seen yet.
recommendations = movies[~movies['MID'].isin(user_full['MID'])] \
                  .merge(pd.DataFrame(sorted_predictions).reset_index(), 
                         how = 'left', left_on = 'MID', right_on = 'MID') \
                  .rename(columns = {user_row: 'Predictions'}) \
                  .sort_values('Predictions', ascending = False) \
                  .iloc[:5, :-1]

In [25]:
recommendations

,MID,Title,Genre
1311,1374,Star Trek: The Wrath of Khan (1982),Action|Adventure|Sci-Fi
1148,1193,One Flew Over the Cuckoo's Nest (1975),Drama
1312,1376,Star Trek IV: The Voyage Home (1986),Action|Adventure|Sci-Fi
285,296,Pulp Fiction (1994),Crime|Drama
570,590,Dances with Wolves (1990),Adventure|Drama|Western


Using above function, we can now get a set of recommendations for any user. 

In [9]:
# Get a list of already reated and recommended movies for a selected user
# Uncomment to run below


rated, recommended = recommender(predictions_df, 100, movies, ratings, 10)

User 100 has already rated 76 movies.
Recommending highest 10 predicted ratings movies not already rated.


In [10]:
# Uncomment to run 

rated.head(10)

,UID,MID,Rating,Time,Title,Genre
1,100,800,5,977593915,Lone Star (1996),Drama|Mystery
63,100,527,5,977594839,Schindler's List (1993),Drama|War
16,100,919,5,977594947,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical
17,100,924,4,977594873,2001: A Space Odyssey (1968),Drama|Mystery|Sci-Fi|Thriller
29,100,969,4,977594044,"African Queen, The (1951)",Action|Adventure|Romance|War
22,100,2406,4,977594142,Romancing the Stone (1984),Action|Adventure|Comedy|Romance
47,100,318,4,977594839,"Shawshank Redemption, The (1994)",Drama
20,100,858,4,977593950,"Godfather, The (1972)",Action|Crime|Drama
49,100,329,4,977594297,Star Trek: Generations (1994),Action|Adventure|Sci-Fi
50,100,260,4,977593595,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi


In [11]:
# Uncomment to run

print ("\nTop Ten recommendations for selected user" )
recommended


Top Ten recommendations for selected user


,MID,Title,Genre
1311,1374,Star Trek: The Wrath of Khan (1982),Action|Adventure|Sci-Fi
1148,1193,One Flew Over the Cuckoo's Nest (1975),Drama
1312,1376,Star Trek IV: The Voyage Home (1986),Action|Adventure|Sci-Fi
285,296,Pulp Fiction (1994),Crime|Drama
570,590,Dances with Wolves (1990),Adventure|Drama|Western
1184,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
877,912,Casablanca (1942),Drama|Romance|War
1161,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller
1524,1617,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
997,1036,Die Hard (1988),Action|Thriller


For above randomly selected user `100`, we can subjectively evaluate that the recommender is doing a decent job. The movies being recommended are quite similar in taste as movies already rated by user. Remember this system is built using only a small subset of the complete MovieLense database which carries potential for significant improvement in predictive performance.  

## Level Up - Optional 

- Run the experiment again using validation testing to identify the optimal value of rank `k`
- Create Test and Train datasets to predict and evaluate the ratings using a suitable method (e.g. RMSE)
- How much of an improvement do you see in recommendations as a result of validation ? 

- Ask other interesting questions
<img src="ideas.jpg" width=400>

## Additional Resources
- https://towardsdatascience.com/how-to-build-a-simple-song-recommender-296fcbc8c85 - A similar system recommending songs to users

- http://surpriselib.com/ Surprise is a Python Scikit is a very popular module for building receommendation systems

## Summary

In this lab, we learned that we can make good recommendations with collaborative filtering methods using latent features from low-rank matrix factorization methods. This technique also scales significantly better to larger datasets. We will next work with a larger MovieLense dataset and using the mapreduce techniques seen in the previous section, we will use PySpark to implement a similar approach using an implementation of matrix factorization called Alternating Least Squares (ALS) method. 